# 01: Load the Coffee sales data from SUEDHANG

#### First create an dataframe with rows for each day, then fill it with the corresponding sales data for that day

In [ ]:
import os
import numpy as np
import pandas as pd
from utils.utils import add_delimiters

# Create a dataframe with rows for every day in the years 2021 to 2022, add additional date columns and 
df = pd.DataFrame(pd.date_range('2021-01-01', '2022-12-31', ), columns=['date'])
df['day_of_week'] = df['date'].dt.day_name()
df['month'] = df['date'].dt.month_name()
df['year'] = df['date'].dt.year
df['week_of_year'] = df['date'].dt.isocalendar().week
df

#### Define the specific names of the relevant rows/columns/variables in the raw data

In [ ]:
category_index = "Unnamed: 0"
sales_date_index = "Unnamed: 1"
total_count_index = "Unnamed: 2"
total_sales_index = "Unnamed: 3"

years = ["2021", "2022"]
months = ["01jan", "02feb", "03mar", "04apr", "05may", "06jun", "07jul", "08aug", "09sep", "10oct", "11nov", "12dec"]

# load a previously manually curated list of coffee products
coffeeproducts_text = open("coffeeproducts.txt",'r', encoding="utf-8").readlines()
coffeeproducts = ['Getränke', 'Coffee']
coffeeproducts.extend(str(row.strip()) for row in coffeeproducts_text)
print("all_categories", coffeeproducts)

coffeeproducts_list_sales = [f"{coffeeproduct}_sales" for coffeeproduct in coffeeproducts]
coffeeproducts_list_count = [f"{coffeeproduct}_count" for coffeeproduct in coffeeproducts]
df[coffeeproducts_list_sales] = 0.0
df[coffeeproducts_list_count] = 0.0

# remove the "total amount" rows from the list of coffee products
coffeeproducts_list_sales = coffeeproducts_list_sales[2:]
coffeeproducts_list_count = coffeeproducts_list_count[2:]
df

#### Load the sales data for all files, write it to its corresponding date into the dataframe

In [ ]:
for year in years:
    for month in months:
        data_dir = os.path.join('raw_coffeesalesdata', year, month)
        data_names = os.listdir(data_dir)
        for day_file in data_names:
            data = pd.read_csv(add_delimiters(os.path.join(data_dir, day_file), delimiter=';'), sep=';', encoding="utf-8")
            data_date = data[sales_date_index][1]
            data_date = pd.to_datetime(data_date, format='%d.%m.%Y').strftime('%Y-%m-%d')
            # merge the main_categories and all_categories together
            for category in coffeeproducts:
                try:
                    df.loc[df['date'] == data_date, f"{category}_sales"] = data.loc[data[category_index] == category, total_sales_index].values[0]
                    df.loc[df['date'] == data_date, f"{category}_count"] = data.loc[data[category_index] == category, total_count_index].values[0]
                except IndexError:
                    #print(f"No {category} sold on {data_date}")
                    pass

# sum up all the total over coffee products
df["Coffee_sales"] = df[coffeeproducts_list_sales].apply(pd.to_numeric, errors='coerce').sum(axis=1)
df["Coffee_count"] = df[coffeeproducts_list_count].apply(pd.to_numeric, errors='coerce').sum(axis=1)

# remove the rows where no coffee was sold
df = df[df["Coffee_sales"] != 0]
df.head()

In [ ]:
# print the top k maximal values of Getränke_sales and the corresponding date
k = 15
var = 'Getränke'
# first convert the sales to float
df.loc[f'{var}_sales'] = df.loc[f'{var}_sales'].astype(float)
df.loc[f'{var}_count'] = df.loc[f'{var}_count'].astype(int)
print(df.nlargest(k, f'{var}_sales')[['date', f'{var}_sales']])
print(df.nlargest(k, f'{var}_count')[['date', f'{var}_count']])

In [ ]:
# Manually correct certain dates, where the sales data for two or more days is entered in one day

vars = ['Getränke', 'Coffee']
sal_cnt = ['sales', 'count']

for var in vars:
    # first convert the sales to float
    df[f'{var}_sales'] = df[f'{var}_sales'].astype(float)
    df[f'{var}_count'] = df[f'{var}_count'].astype(int)

    for sal in sal_cnt:
        # 2021-10-16 has all data for 2021-10-16 and 2021-10-17,
        print(df.loc[df["date"] == "2021-10-16", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2021-10-17", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2021-10-16", f'{var}_{sal}'].values[0]
        getr_sales_on_2021_10_16 = combined_sales * 0.58
        getr_sales_on_2021_10_17 = combined_sales - getr_sales_on_2021_10_16
        df.loc[df['date'] == "2021-10-16", f'{var}_{sal}'] = getr_sales_on_2021_10_16
        df.loc[df['date'] == "2021-10-17", f'{var}_{sal}'] = getr_sales_on_2021_10_17
        print(df.loc[df["date"] == "2021-10-16", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2022-02-13 has all data for 2022-02-13 and 2022-02-14
        print(df.loc[df["date"] == "2022-02-13", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2022-02-14", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2022-02-13", f'{var}_{sal}'].values[0]
        getr_sales_on_2022_02_13 = combined_sales * 0.739
        getr_sales_on_2022_02_14 = combined_sales - getr_sales_on_2022_02_13
        df.loc[df['date'] == "2022-02-13", f'{var}_{sal}'] = getr_sales_on_2022_02_13
        df.loc[df['date'] == "2022-02-14", f'{var}_{sal}'] = getr_sales_on_2022_02_14
        print(df.loc[df["date"] == "2022-02-13", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2021-07-31 has all data for 2021-07-31 and 2021-08-01
        print(df.loc[df["date"] == "2021-07-31", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2021-08-01", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2021-07-31", f'{var}_{sal}'].values[0]
        getr_sales_on_2021_07_31 = combined_sales * 0.618
        getr_sales_on_2021_08_01 = combined_sales - getr_sales_on_2021_07_31
        df.loc[df['date'] == "2021-07-31", f'{var}_{sal}'] = getr_sales_on_2021_07_31
        df.loc[df['date'] == "2021-08-01", f'{var}_{sal}'] = getr_sales_on_2021_08_01
        print(df.loc[df["date"] == "2021-07-31", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2022-01-08 has all data for 2022-01-08 and 2022-01-09
        print(df.loc[df["date"] == "2022-01-08", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2022-01-09", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2022-01-08", f'{var}_{sal}'].values[0]
        getr_sales_on_2022_01_08 = combined_sales * 0.523
        getr_sales_on_2022_01_09 = combined_sales - getr_sales_on_2022_01_08
        df.loc[df['date'] == "2022-01-08", f'{var}_{sal}'] = getr_sales_on_2022_01_08
        df.loc[df['date'] == "2022-01-09", f'{var}_{sal}'] = getr_sales_on_2022_01_09
        print(df.loc[df["date"] == "2022-01-08", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2022-07-25 has all data for 2022-07-25 and 2022-07-26
        print(df.loc[df["date"] == "2022-07-25", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2022-07-26", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2022-07-25", f'{var}_{sal}'].values[0]
        getr_sales_on_2022_07_25 = combined_sales * 0.469
        getr_sales_on_2022_07_26 = combined_sales - getr_sales_on_2022_07_25
        df.loc[df['date'] == "2022-07-25", f'{var}_{sal}'] = getr_sales_on_2022_07_25
        df.loc[df['date'] == "2022-07-26", f'{var}_{sal}'] = getr_sales_on_2022_07_26
        print(df.loc[df["date"] == "2022-07-25", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2022-08-11 has all data for 2022-08-11 and 2022-08-12
        print(df.loc[df["date"] == "2022-08-11", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2022-08-12", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2022-08-11", f'{var}_{sal}'].values[0]
        getr_sales_on_2022_08_11 = combined_sales * 0.486
        getr_sales_on_2022_08_12 = combined_sales - getr_sales_on_2022_08_11
        df.loc[df['date'] == "2022-08-11", f'{var}_{sal}'] = getr_sales_on_2022_08_11
        df.loc[df['date'] == "2022-08-12", f'{var}_{sal}'] = getr_sales_on_2022_08_12
        print(df.loc[df["date"] == "2022-08-11", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2021-12-13 has all data for 2021-12-13 and 2021-12-14 and 2021-12-15
        print(df.loc[df["date"] == "2021-12-13", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2021-12-14", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2021-12-15", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2021-12-13", f'{var}_{sal}'].values[0]
        getr_sales_on_2021_12_13 = combined_sales * 0.2895
        getr_sales_on_2021_12_14 = combined_sales * 0.377
        getr_sales_on_2021_12_15 = combined_sales - getr_sales_on_2021_12_13 - getr_sales_on_2021_12_14
        df.loc[df['date'] == "2021-12-13", f'{var}_{sal}'] = getr_sales_on_2021_12_13
        df.loc[df['date'] == "2021-12-14", f'{var}_{sal}'] = getr_sales_on_2021_12_14
        df.loc[df['date'] == "2021-12-15", f'{var}_{sal}'] = getr_sales_on_2021_12_15
        print(df.loc[df["date"] == "2021-12-13", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2021-12-22 has all data for 2021-12-22 and 2021-12-23
        print(df.loc[df["date"] == "2021-12-22", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2021-12-23", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2021-12-22", f'{var}_{sal}'].values[0]
        getr_sales_on_2021_12_22 = combined_sales * 0.516
        getr_sales_on_2021_12_23 = combined_sales - getr_sales_on_2021_12_22
        df.loc[df['date'] == "2021-12-22", f'{var}_{sal}'] = getr_sales_on_2021_12_22
        df.loc[df['date'] == "2021-12-23", f'{var}_{sal}'] = getr_sales_on_2021_12_23
        print(df.loc[df["date"] == "2021-12-22", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")

        # 2021-11-09 has all data for 2021-11-09 and 2021-11-10
        print(df.loc[df["date"] == "2021-11-09", ["date", f'{var}_{sal}']])
        print(df.loc[df["date"] == "2021-11-10", ["date", f'{var}_{sal}']])
        combined_sales = df.loc[df["date"] == "2021-11-09", f'{var}_{sal}'].values[0]
        getr_sales_on_2021_11_09 = combined_sales * 0.546
        getr_sales_on_2021_11_10 = combined_sales - getr_sales_on_2021_11_09
        df.loc[df['date'] == "2021-11-09", f'{var}_{sal}'] = getr_sales_on_2021_11_09
        df.loc[df['date'] == "2021-11-10", f'{var}_{sal}'] = getr_sales_on_2021_11_10
        print(df.loc[df["date"] == "2021-11-09", ["date", f'{var}_{sal}']])
        print("\n ---------------------------- \n")
    
    df[f'{var}_count'] = df[f'{var}_count'].astype(int)


In [ ]:
df.info()

In [ ]:
# drop all rows where no sales were made/are not available
#df = df[df["Getränke_sales"] != 0]
# save the df as a csv file
df.to_csv('processed_data/sales_2021-2022.csv', index=False)